### Noms et Prénoms du binome:

### Veulliez à rendre le TP sous format PDF
---

# TP en Watermarking et Stéganographie

---

## Patrick Bas, CNRS, CRIStAL

---


 

## Tatouage par également de spectre et attaques de sécurité

### Notations et rappels:

Les notations sont identiques à celles vues en cours. Le procédé d'insertion est le schéma par étalement de spectre vu en cours.

* X = matrice de $N_{i}$ vecteurs originaux de taille $N_{v}$ ($N_{i}$ colonnes, $N_{v}$ lignes). $N_{i}$ représente par exemple le nombre de contenu traités, et $N_{v}$ le nombre de composantes tatouées par contenu. **Note**: chaque colonne de X peut par exemple représenter des composantes d'une image.
* $N_{o}$ représente le nombre de contenus tatoués observés par l'adversaire et utilisés pour construire son attaque
* $N_{i}$ représente le nombre de contenus tatoués utilisés pour calculer pratiquement le taux d'erreur (voir BER)
* Y = matrice de contenus tatoués
* Z = matrice de contenus tatoués et perturbés
* k clé secrète de norme unitaire
* $m_{1}$: bit inséré, converti en +1, -1 
* $\alpha$: paramètre de distorsion
* BER: Bit Error Rate, taux d'erreur binaire ou encore probabilité d'erreur empirique de décodage

#### Rappels:
* L'objectif du récepteur et de bien décoder $m_{1}$, possiblement en ayant une distortion qui ne soit pas trop importante
* Ici, les objectifs de l'adversaire sont d'estimer une version k ou +/-k puis d'effacer le message inséré. Pour s'assurer que l'adversaire a bien réussi à estimer k ou +/-k, il calculera la correlation normalisée entre le vecteur k et son estimation.

**N'oubliez pas d'exporter votre TP en pdf de préférence ou éventuellement en html lors de sa remise**



In [1]:
%matplotlib inline  

import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erf
from sklearn.decomposition import FastICA

## Scénario 1: Attaque à Messages connus, 1 bit
* Mise en route: Quel est le BER cible de l'adversaire ?
* Mettre en place l'attaque liée à ce scénario
* Etudier l'impact de $N_{o}$ et de $\alpha$ sur le BER après attaque

In [2]:
Nv = 100 # Size of the vector
Ni = 10000 # Max number of observations

In [ ]:
def ber(Y,m,k):#Compute the Bit Error Rate between message m and the extracted message from Y using key k
    c = np.sign(np.dot(Y.T,k))
    return np.sum(c != m)/np.float(Ni)

def norm_corr(hatk,k):
    hatk = hatk / np.sqrt(np.dot(hatk.T,hatk)) # Normalize
    corrN = np.abs(np.dot(hatk.T,k))/(np.linalg.norm(hatk)*np.linalg.norm(k)) # Compute the Normalised correlation
    return corrN

def do_process(alpha,No):
    
    print('alpha: ',alpha)
    print('No: ',No)

    X = np.random.randn(Nv,Ni) # Generate Ni random host vectors
    k = np.random.randn(Nv,1) # Generate de Watermark
    k = k / np.sqrt(np.dot(k.T,k)) # Normalize the watermark

    m1 = np.ones((Ni,1)) #Scenario with Known Messages: generate only ones

    K = np.dot(k,m1.T) # Generate the matrix of watermarks (each column contains m1_i*k)

    W = alpha*K

    Y = X + W # perform embedding


    cY = np.sign(np.dot(Y.T,k)) # Computation of the decoded 'bits' (here -1 or +1)
    #print('practical bit error rate:')
    #print(np.sum(cY != m1)/np.float(Ni)) 

    Y_obs = Y[:,:No]
    # To Do: implement the attack on No observation, i.e. on Y[:,:No]
    
    # hatk = ???
    hatk = np.zeros(Nv)# To change
    hatk = hatk / np.sqrt(np.dot(hatk.T,hatk)) # We need to Normalize
    
    corrN = np.abs(np.dot(hatk.T,k))/(np.linalg.norm(hatk)*np.linalg.norm(k)) # Compute the Normalised correlation
    print('Normalised correlation between the true key and the estimated key')
    print(corrN)

    hatk = np.reshape(hatk,(Nv,1))# We need to reshape

    YA = Y - alpha*np.dot(hatk,m1.T) # perform the removal attack
    print('practical bit error rate after security attack')
    print(ber(YA,m1,k))
    print('\n\n')
    

alpha = 2 # Tune the power of the watermark here

# One example, can be used to draw plots
for No in range(100,2000,100):
    do_process(alpha,No)

## Scénario 2, Attaque à messages inconnus, 1bit
* Note: la fonction `np.linalg.eig` peut être utilisée pour effectuer une décomposition en valeurs et vecteurs propres.
* Note: pour effacer le message inséré, il conviendra au préalable d'estimer le "bit" inséré (m=-1 ou m=1), cela peut se faire via `m_est = np.sign(np.dot(Y.T,hatk))` où `hatk` est la clé estimée
* Mettre en place l'attaque
* Etudier l'impact de $N_{o}$ et de $\alpha$ sur le BER après attaque

In [ ]:
# 2dn scenario, scenario with KMA 
print('2nd Scenario')

def do_process(alpha,No):
    
    print('alpha: ',alpha)
    print('No: ',No)

    X = np.random.randn(Nv,Ni) # Generate Ni random host vectors
    k = np.random.randn(Nv,1) # Generate de Watermark
    k = k / np.sqrt(np.dot(k.T,k)) # Normalize the watermark

    m1 = np.sign(np.random.randn(Ni,1)) #Scenario with unknow messages, first bit

    K = np.dot(k,m1.T) # Generate the matrix of watermarks (each column contains m1_i*k)

    W = alpha*K

    Y = X + W # perform embedding

    Y_obs = Y[:,:No]
    #hatk = TO DO
    hatk = np.zeros(Nv)# To change
    
    hatk = np.reshape(hatk,(Nv,1)) #You might need to reshape the estimated key

    corrN = norm_corr(hatk,k)# To ease the writing we use the norm_corr function

    print('Normalised correlation between the true key and the estimated key')
    print(corrN)

    #m_est = TO DO
    #YA = TO DO 
    YA = np.zeros((Nv,Ni))# To change

    print('bit error rate after security attack')
    print(ber(YA,m1,k))
    print('\n\n')


    
alpha = 2 # Tune the power of the watermark here

# One example, can be used to draw plots
for No in range(100,2000,100):
    do_process(alpha,No)

## Scénario 3: Attaque à messages inconnus, 2 bits
* Vérifier que l'attaque précédente ne permet pas d'estimer les deux clés. 
* Estimer au moins l'une des clés utilisée
    * Note: on pourra utiliser l'algorithme `FastICA` pour estimer les deux composantes indépendantes 
    * Pour cela on pourra appeler la fonction fastica en spécifiant que l'analyse en composantes indépendantes s'effectuera sur un sous espace engendré par les *deux premières composantes principales* (`n_components=2`), et en récupérant les colonnes de la matrice de mélange A (obtenue via `ica.mixing_`) estimé par l'algorithme.
    * Vérifier, à l'aide de la corrélation normalisée, que cette méthode permet d'estimer la clé `k1`
* Mettre en place l'attaque qui permet les deux bits
* Etudier l'impact de $N_{o}$ et de $\alpha$ sur le BER après attaque

In [ ]:
# 3rd scenario, 2 bits
print('3rd scenario, 2bits')

m1 = np.sign(np.random.randn(Ni,1))#Scenario with unknow messages, first bit
m2 = np.sign(np.random.randn(Ni,1))#Scenario with unknow messages, second bit

k1 = np.random.randn(Nv,1) # Generate de Watermark
k1 = k1 / np.sqrt(np.dot(k1.T,k1)) # Normalize the watermark

k2 = np.random.randn(Nv,1) # Generate de Watermark
k2 = k2 / np.sqrt(np.dot(k2.T,k2)) # Normalize the watermark

def do_process(alpha,No):
    
    print('alpha: ',alpha)
    print('No: ',No)

    X = np.random.randn(Nv,Ni) # Generate Ni random host vectors

    K = np.dot(k1,m1.T) + np.dot(k2,m2.T) # Generate the matrix of watermarks (each column contains m1_i*k)

    W = alpha*K

    Y = X + W # perform embedding

    
    Y_obs = Y[:,:No]
    # TO DO
    
    # hatk_1 = TO DO
    hatk_1 = np.zeros(Nv)# To change
    
    hatk_1 = hatk_1/np.linalg.norm(hatk_1)
    hatk_1 = np.reshape(hatk_1,(Nv,1))

    corrN_1 = norm_corr(hatk_1,k1) # Compute the Normalised correlation
    print('Normalised correlation between the estimated key and k1')
    print(corrN_1)

    corrN_2 = norm_corr(hatk_1,k2) # Compute the Normalised correlation
    print('Normalised correlation between the estimated key and k2')
    print(corrN_2)

    # Perform the attack
    # To do
    YA_1 = np.zeros((Nv,Ni))# To change   
    
    
    print('bit error rate after security attack for the first bit')
    print(ber(YA_1,m1,k1))

    print('bit error rate after security attack for the second bit')
    print(ber(YA_1,m2,k2))
    print('\n\n')



alpha = 2 # Tune the power of the watermark here

# One example, can be used to draw plots
for No in range(100,2000,100):
    do_process(alpha,No)

## Etude de la robustesse
* Calculer le taux d'erreur (BER pour Bit Error Rate) théorique après ajout de bruit (voir cours)
* Etudier l'évolution de la robustesse (via le BER) en fonction de la distortion (via le paramètre $\alpha$) 


## Conclusions
* Pourquoi un compromis entre la robustesse et la sécurité est-il inévitable ?